In [30]:
import pandas as pd
import os

class DriverTrustEngine:
    def __init__(self):
        self.drivers_file = "drivers_2026.xlsx"
        self.orders_file = "delivery_orders_2026.xlsx"

    def calculate_reliability(self, row):
        """기사 한 명의 신뢰도 점수를 산출하는 로직"""
        # 1. 기초 데이터 추출 (데이터가 없을 경우 기본값 설정)
        try:
            accidents = int(row.get('사고건수', 0))
            agency_score = float(row.get('대리점평점', 80))  # 0~100점 기준
            customer_score = float(row.get('고객평점', 80))  # 0~100점 기준
            total_trips = int(row.get('완료횟수', 0))
        except (ValueError, TypeError):
            return 0.0

        # 2. 신뢰도 계산 (가중치 적용)
        # 평점 기반 기본 신뢰도 (평균 점수 개념)
        base_score = (agency_score * 0.35) + (customer_score * 0.65)
        
        # 3. 감점 및 가점 적용
        # 사고 감점: 사고 1건당 30점씩 대폭 감점
        penalty = accidents * 30
        
        # 숙련도 가점: 100건 이상 완료 시 5점 추가
        bonus = 5 if total_trips >= 100 else 0
        
        # 최종 점수 산출 (최저 0점 ~ 최고 100점 제한)
        final_score = base_score - penalty + bonus
        return max(0.0, min(100.0, round(final_score, 2)))

    def update_driver_ranks(self):
        """기사 목록 전체의 신뢰도를 갱신하고 순위를 매김"""
        if not os.path.exists(self.drivers_file):
            print("기사 데이터 파일이 없습니다.")
            return None

        df = pd.read_excel(self.drivers_file)
        
        # 신뢰도 점수 계산 적용
        df['신뢰도점수'] = df.apply(self.calculate_reliability, axis=1)
        
        # 신뢰도 등급 부여 (2026년형 등급제)
        def get_grade(score):
            if score >= 90: return "S (최우수)"
            elif score >= 80: return "A (우수)"
            elif score >= 60: return "B (보통)"
            else: return "C (관리대상)"
            
        df['신뢰도등급'] = df['신뢰도점수'].apply(get_grade)
        
        # 점수 순으로 정렬하여 다시 저장
        df = df.sort_values(by='신뢰도점수', ascending=False)
        df.to_excel(self.drivers_file, index=False)
        return df

    def match_best_driver(self, order_id):
        """배차 실행: 신뢰도 1위 기사 자동 매칭"""
        df_drivers = self.update_driver_ranks()
        if df_drivers is None: return "기사 데이터 로드 실패"

        # 보험 가입자 중 가장 신뢰도가 높은 기사 선택
        eligible_drivers = df_drivers[df_drivers["탁송보험"] == "가입"]
        
        if eligible_drivers.empty:
            return "현재 배차 가능한 보험 가입 기사가 없습니다."

        best_driver = eligible_drivers.iloc[0] # 정렬된 첫 번째 기사
        
        return {
            "기사명": best_driver["이름"],
            "신뢰도점수": best_driver["신뢰도점수"],
            "등급": best_driver["신뢰도등급"],
            "사고이력": f"{best_driver.get('사고건수', 0)}건",
            "연락처": best_driver["연락처"]
        }

# --- 실행부 ---
if __name__ == "__main__":
    engine = DriverTrustEngine()
    
    # 1. 모든 기사 신뢰도 갱신 및 결과 확인
    ranked_list = engine.update_driver_ranks()

    print("📋 [기사 신뢰도 순위 리스트]")
    print(ranked_list[['이름', '신뢰도점수', '신뢰도등급']].head())

# --- 매칭 정보 출력 로직 추가 ---

# 예시: 가장 상위 순위의 기사를 매칭했다고 가정 (인덱스 999)
matched_index = 999 

if matched_index in ranked_list.index:
    # 해당 인덱스(999)에 해당하는 기사 정보 추출
    matched_driver = ranked_list.loc[matched_index]
    
    # 출력 데이터 준비
    driver_name = matched_driver['이름']
    driver_contact = matched_driver['연락처'] # 이미지 기준 '연락처' 컬럼 사용

    print(f"\n--- 🎫 매칭 정보 (2026-01-12 14:52:00) ---")
    print(f"📍 탁송 출발지: 서울시 337번지 인근 (37.5665, 126.978)")
    print(f"👤 매칭된 기사: {driver_name}")
    print(f"🔢 데이터프레임 인덱스: **{matched_index}**")
    print(f"📞 출발지/기사 연락처: **{driver_contact}**")
    print("------------------------------------------")
else:
    print(f"\n오류: 인덱스 {matched_index}에 해당하는 기사를 찾을 수 없습니다.")    

    # 2. 자동 배차 시뮬레이션
    match = engine.match_best_driver(order_id=1)
    if isinstance(match, dict):
        print(f"\n🎯 [자동 배차 완료]")
        print(f"배정 기사: {match['기사명']} ({match['등급']})")
        print(f"매칭 근거: 신뢰도 점수 {match['신뢰도점수']}점 / 사고 {match['사고이력']}")

📋 [기사 신뢰도 순위 리스트]
      이름  신뢰도점수   신뢰도등급
999  장성우   80.0  A (우수)
0    정하윤   80.0  A (우수)
1    김민준   80.0  A (우수)
2    김민준   80.0  A (우수)
3    박도현   80.0  A (우수)

--- 🎫 매칭 정보 (2026-01-12 14:52:00) ---
📍 탁송 출발지: 서울시 337번지 인근 (37.5665, 126.978)
👤 매칭된 기사: 장성우
🔢 데이터프레임 인덱스: **999**
📞 출발지/기사 연락처: **010-1876-7154**
------------------------------------------
